In [1]:
import numpy as np
import math
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.DataStructs.cDataStructs import ConvertToNumpyArray
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [101]:

df = pd.read_csv('../data/interim/DATASET.csv', delimiter=';')
df = df[:155]
fp_array = np.zeros((0,), dtype=np.int8) #создаем массив из нулей
for i in range(len(df['Reaction SMILES'])): #итерируемся по столбцу со smiles реакций
  r = df['Reaction SMILES'].loc[i].split('>>') #разделяем smiles реакции на smiles реагентов и smiles продуктов
  reactants = r[0].split('.') #разделяем общий smiles реагентов на smiles отдельных молекул реагентов
  products = r[1].split('.') #разделяем общий smiles продуктов на smiles отдельных молекул продуктов
  for j in reactants:
    # print(j)
    r_mol = Chem.MolFromSmiles(j) #превращаем smiles в молекулу rdkit
    r_fp = AllChem.GetMorganFingerprintAsBitVect(r_mol, 2, nBits=32) #получаем fingerprint с радиусом=2 из молекулы
    ConvertToNumpyArray(r_fp, fp_array) #превращаем fingerprint в массив
    # print(fp_array) #это итоговый фингерпринт в виде массива
  for k in products: #то же самое для продуктов
    p_mol = Chem.MolFromSmiles(k)
    p_fp = AllChem.GetMorganFingerprintAsBitVect(p_mol, 2, nBits=32)
    ConvertToNumpyArray(p_fp, fp_array)
    #print(fp_array)

# #Дальше тут все то же самое для столбцов с условиями реакций
#Это все можно запихнуть в один цикл, но я не знаю в каком виде это нужно будет для модели, поэтому пока так сделала
for i in range(len(df['Reagent1_SMILES'])):  #итерируемся по столбцу реагентов
  if type(df['Reagent1_SMILES'].loc[i]) != float: #пропускаем пустые ячейки
    reag1_mol = Chem.MolFromSmiles(df['Reagent1_SMILES'].loc[i]) #превращаем smiles в молекулу
    reag1_fp = AllChem.GetMorganFingerprintAsBitVect(reag1_mol, 2, nBits=32) #получаем fingerprint из молекулы
    ConvertToNumpyArray(reag1_fp, fp_array) #превращаем fingerprint в массив
    # print(len(fp_array)) #это итоговый фингерпринт реагента в виде массива

for i in range(len(df['Reagent2_SMILES'])):  
  if type(df['Reagent2_SMILES'].loc[i]) != float:
    reag2_mol = Chem.MolFromSmiles(df['Reagent2_SMILES'].loc[i])
    reag2_fp = AllChem.GetMorganFingerprintAsBitVect(reag2_mol, 2, nBits=32)
    ConvertToNumpyArray(reag2_fp, fp_array)
    #print(fp_array)
     

for i in range(len(df['Solvent1_SMILES'])):  
  if type(df['Solvent1_SMILES'].loc[i]) != float:
    solv1_mol = Chem.MolFromSmiles(df['Solvent1_SMILES'].loc[i])
    solv1_fp = AllChem.GetMorganFingerprintAsBitVect(solv1_mol, 2, nBits=32)
    ConvertToNumpyArray(solv1_fp, fp_array)
    #print(fp_array)
    # print(len(fp_array))

for i in range(len(df['Solvent2_SMILES'])):  
  if type(df['Solvent2_SMILES'].loc[i]) != float:
    solv2_mol = Chem.MolFromSmiles(df['Solvent2_SMILES'].loc[i])
    solv2_fp = AllChem.GetMorganFingerprintAsBitVect(solv2_mol, 2, nBits=32)
    ConvertToNumpyArray(solv2_fp, fp_array)
    #print(fp_array)

for i in range(len(df['Catalyst_SMILES'])):  
  if type(df['Catalyst_SMILES'].loc[i]) != float:
    cat_mol = Chem.MolFromSmiles(df['Catalyst_SMILES'].loc[i])
    cat_fp = AllChem.GetMorganFingerprintAsBitVect(cat_mol, 2, nBits=32)
    ConvertToNumpyArray(cat_fp, fp_array)
    #print(fp_array)
    # print(len(fp_array))

[14:42:58] WARNING: not removing hydrogen atom without neighbors
[14:42:58] WARNING: not removing hydrogen atom without neighbors
[14:42:58] WARNING: not removing hydrogen atom without neighbors


In [174]:
dataset = pd.read_csv('../data/interim/DATASET.csv', delimiter=';')
dataset = dataset[:154]
dataset = dataset[~(dataset['Reaction ID'].isin([55883694, 23404571, 25406145, 22959501, 24137279]))]

In [175]:
dataset

,Reaction ID,Reactant,Product,Reaction SMILES,Fulltext of reaction,Number of Stages,Time (Reaction Details) [h],Temperature (Reaction Details) [C],Pressure (Reaction Details) [Torr],pH-Value (Reaction Details),...,Reagent1_SMILES,Reagent_2,Reagent2_SMILES,Solvent_1,Solvent1_SMILES,Solvent_2,Solvent2_SMILES,Catalyst,Catalyst_SMILES,Unnamed: 21
0,5285945.0,para-nitrophenyl bromide,5-bromo-2-nitroaniline,BrC1=CC=C(C=C1)N(=O)=O>>NC1=CC(Br)=CC=C1N(=O)=O,Example 14a; 7V-(2-Amino-5-(lH-pyrrol-2-yl)phe...,NaN,4.83333,10.0,NaN,NaN,...,CON,Potassium tert-butoxide,[K+].CC(C)(C)[O-],"1,2-dimethoxyethane",COCCOC,"N,N-dimethyl-formamide",CN(C)C=O,copper(l) chloride,[Cl-].[Cu+],NaN
1,5285945.0,para-nitrophenyl bromide,5-bromo-2-nitroaniline,BrC1=CC=C(C=C1)N(=O)=O>>NC1=CC(Br)=CC=C1N(=O)=O,NaN,NaN,NaN,20.0,NaN,NaN,...,CON,Potassium tert-butoxide,[K+].CC(C)(C)[O-],"N,N-dimethyl-formamide",CN(C)C=O,NaN,NaN,copper(l) chloride,[Cl-].[Cu+],NaN
2,727677.0,"1,2-naphthoquinone","4-amino-1,2-naphthoquinone",O=C1C=CC2=C(C=CC=C2)C1=O>>NC1=CC(=O)C(=O)C2=CC...,"In brief, to a stirred suspension of compound ...",NaN,NaN,NaN,NaN,NaN,...,[Na+].[N-]=[N+]=[N-],acetic acid,CC(O)=O,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,727677.0,"1,2-naphthoquinone","4-amino-1,2-naphthoquinone",O=C1C=CC2=C(C=CC=C2)C1=O>>NC1=CC(=O)C(=O)C2=CC...,NaN,NaN,2.00000,80.0,NaN,NaN,...,C[Si](C)(C)N=[N+]=[N-],NaN,NaN,"N,N-dimethyl-formamide",CN(C)C=O,NaN,NaN,NaN,NaN,NaN
4,727677.0,"1,2-naphthoquinone","4-amino-1,2-naphthoquinone",O=C1C=CC2=C(C=CC=C2)C1=O>>NC1=CC(=O)C(=O)C2=CC...,NaN,NaN,2.00000,40.0,NaN,NaN,...,[Na+].[N-]=[N+]=[N-],NaN,NaN,water,O,acetic acid,CC(O)=O,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,49993643.0,"4,4-diethyl-2,2-dimethylchroman","4,4-diethyl-2,2-dimethylchroman-6-amine",CCC1(CC)CC(C)(C)OC2=C1C=CC=C2>>CCC1(CC)CC(C)(C...,"General procedure: A 25-mL, three-necked, roun...",2.0,2.00000,85.0,NaN,NaN,...,O[N+]([O-])=O,acetic anhydride,CC(=O)OC(C)=O,ethanol,CCO,water,O,iron,[Fe],NaN
150,53956135.0,"1,1,3,3-tetraethyl-5-nitroisoindoline","5-amino-6-nitro-1,1,3,3-tetraethylisoindoline",CCC1(CC)NC(CC)(CC)C2=C1C=CC(=C2)N(=O)=O>>CCC1(...,NaN,NaN,40.00000,23.0,NaN,NaN,...,[K+].CC(C)(C)[O-],"N,N,N-trimethylhydrazinium iodide",[I-].C[N+](C)(C)N,dimethyl sulfoxide,C[S](C)=O,NaN,NaN,NaN,NaN,NaN
151,53956135.0,"1,1,3,3-tetraethyl-5-nitroisoindoline","5-amino-6-nitro-1,1,3,3-tetraethylisoindoline",CCC1(CC)NC(CC)(CC)C2=C1C=CC(=C2)N(=O)=O>>CCC1(...,NaN,NaN,14.00000,NaN,NaN,NaN,...,[K+].CC(C)(C)[O-],"N,N,N-trimethylhydrazinium iodide",[I-].C[N+](C)(C)N,dimethyl sulfoxide,C[S](C)=O,NaN,NaN,NaN,NaN,NaN
152,57338413.0,methyl 4-(picolinamido)benzoate,methyl 3-amino-4-(picolinamido)benzoate,COC(=O)C1=CC=C(NC(=O)C2=CC=CC=N2)C=C1>>COC(=O)...,General procedure: A solution of 2-picolinamid...,2.0,16.00000,80.0,NaN,NaN,...,N=C(c1ccccc1)c2ccccc2,hydrogenchloride,Cl,toluene,Cc1ccccc1,tetrahydrofuran,C1CCOC1,copper(l) chloride,[Cl-].[Cu+],NaN


In [157]:
test_size = 0.2
train_mask = np.random.rand(dataset.shape[0]) > test_size
test_mask = ~train_mask

In [158]:
splited_reaction = dataset['Reaction SMILES'].str.split('>>')

In [159]:
new_df = pd.DataFrame()
new_df[['Reactants', 'Products']] = splited_reaction.tolist()


In [160]:
new_df

,Reactants,Products
0,BrC1=CC=C(C=C1)N(=O)=O,NC1=CC(Br)=CC=C1N(=O)=O
1,BrC1=CC=C(C=C1)N(=O)=O,NC1=CC(Br)=CC=C1N(=O)=O
2,O=C1C=CC2=C(C=CC=C2)C1=O,NC1=CC(=O)C(=O)C2=CC=CC=C12
3,O=C1C=CC2=C(C=CC=C2)C1=O,NC1=CC(=O)C(=O)C2=CC=CC=C12
4,O=C1C=CC2=C(C=CC=C2)C1=O,NC1=CC(=O)C(=O)C2=CC=CC=C12
...,...,...
144,CCC1(CC)CC(C)(C)OC2=C1C=CC=C2,CCC1(CC)CC(C)(C)OC2=C1C=C(N)C=C2
145,CCC1(CC)NC(CC)(CC)C2=C1C=CC(=C2)N(=O)=O,CCC1(CC)NC(CC)(CC)C2=CC(=C(N)C=C12)N(=O)=O
146,CCC1(CC)NC(CC)(CC)C2=C1C=CC(=C2)N(=O)=O,CCC1(CC)NC(CC)(CC)C2=CC(=C(N)C=C12)N(=O)=O
147,COC(=O)C1=CC=C(NC(=O)C2=CC=CC=N2)C=C1,COC(=O)C1=CC(N)=C(NC(=O)C2=NC=CC=C2)C=C1


In [207]:
reactants = new_df['Reactants'].str.split('.')
df_reactants = pd.DataFrame(reactants.values.tolist(), reactants.index).add_prefix('Reagent_')

In [162]:
# products = new_df['Products'].str.split('.')
# df_products = pd.DataFrame(products.values.tolist(), products.index).add_prefix('Products_')
# products

In [163]:
nBits=128

In [164]:
def get_fingerprint(x):
    fp_array = np.zeros((0,), dtype=np.int8)
    if x is not None and type(x) != float:
        mol = Chem.MolFromSmiles(x)
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=nBits) 
        ConvertToNumpyArray(fp, fp_array)
        return fp_array
    else:
        return np.zeros((nBits,), dtype=np.int8)

In [205]:
# df_products = df_products.applymap(get_fingerprint)
df_products = new_df['Products'].apply(get_fingerprint).to_frame().set_index(dataset.index)

In [208]:
df_reactants = df_reactants.applymap(get_fingerprint).set_index(dataset.index)

In [195]:
cols = ['Reagent1_SMILES', 'Reagent2_SMILES', 'Solvent1_SMILES', 'Solvent2_SMILES', 'Catalyst_SMILES']
df_added = dataset[cols].applymap(get_fingerprint)

[17:20:04] WARNING: not removing hydrogen atom without neighbors
[17:20:04] WARNING: not removing hydrogen atom without neighbors
[17:20:04] WARNING: not removing hydrogen atom without neighbors


In [256]:
final_dataset = pd.concat([df_reactants,  df_products, df_added], axis=1)
final_dataset = final_dataset[final_dataset.columns].apply(np.concatenate, axis=1)

In [169]:
# df_cond = pd.DataFrame()
# df_cond['Temperature'] = dataset['Temperature (Reaction Details) [C]'].fillna(20.0)
# df_cond['Pressure'] = dataset['Pressure (Reaction Details) [Torr]'].fillna(760.051)
# df_cond

In [210]:
temp_series = dataset['Temperature (Reaction Details) [C]'].fillna(20.0)

In [211]:
scaler = MinMaxScaler()
temp_series[train_mask] = scaler.fit_transform(temp_series[train_mask].to_numpy().reshape(-1, 1)).reshape(-1,)
temp_series[test_mask] = scaler.transform(temp_series[test_mask].to_numpy().reshape(-1, 1)).reshape(-1,)

In [257]:
final_dataset = pd.concat([final_dataset, temp_series], axis=1)
final_dataset = final_dataset[final_dataset.columns].apply(np.hstack, axis=1).to_frame()
final_dataset['Yield'] = dataset['Yield (numerical)']
final_dataset = final_dataset.rename(columns={0: 'fingerprint'})

In [423]:
train['fingerprint'].iloc[105].shape


(1025,)

In [379]:
train = final_dataset[train_mask]
test = final_dataset[test_mask]

train.to_json('../data/processed/train.csv')
test.to_json('../data/processed/test.csv')

In [42]:
# preprocess -> save dataset train and test
import torch
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torch import nn
from torch.utils.data import DataLoader
from torch.autograd import Variable

In [43]:
class CustomImageDataset(Dataset):
    def __init__(self, data_dir, transform=False):
        self.data = pd.read_json(data_dir)
        self.transform = transform
        if self.transform is True:
            self.X = torch.from_numpy(np.stack(self.data.fingerprint.apply(lambda x: np.array(x)).to_numpy()).astype(np.float64))
        else: 
            self.X = self.data.fingerprint
        if self.transform is True:
            self.y = torch.from_numpy(self.data.Yield.to_numpy().astype(np.float64))
        else: 
            self.y = self.data.Yield

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [44]:
training_data = CustomImageDataset(
    data_dir="../data/processed/train.csv",
    transform=True
)

test_data = CustomImageDataset(
    data_dir="../data/processed/test.csv",
    transform=True
)

In [45]:
type(training_data.y)

torch.Tensor

In [55]:
loss_fn = nn.MSELoss()
learning_rate = 1e-3
batch_size = 12
epochs = 100

In [56]:
type(training_data.X)

torch.Tensor

In [57]:
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [58]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(1025, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [59]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [60]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    
    for batch, (X, y) in enumerate(dataloader):
        X = Variable(X).float()
        y = Variable(y).float()
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = Variable(X).float()
            y = Variable(y).float()
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [61]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 3731.949219  [    0/  118]
Test Error: 
 Accuracy: 0.0%, Avg loss: 2560.365397 

Epoch 2
-------------------------------
loss: 2802.870605  [    0/  118]
Test Error: 
 Accuracy: 0.0%, Avg loss: 3829.395752 

Epoch 3
-------------------------------
loss: 5188.479980  [    0/  118]
Test Error: 
 Accuracy: 0.0%, Avg loss: 3396.250651 

Epoch 4
-------------------------------
loss: 3804.810303  [    0/  118]
Test Error: 
 Accuracy: 0.0%, Avg loss: 3637.417480 

Epoch 5
-------------------------------
loss: 3515.881836  [    0/  118]
Test Error: 
 Accuracy: 0.0%, Avg loss: 3456.281331 

Epoch 6
-------------------------------
loss: 3657.487305  [    0/  118]
Test Error: 
 Accuracy: 0.0%, Avg loss: 3223.334066 

Epoch 7
-------------------------------
loss: 3185.900391  [    0/  118]
Test Error: 
 Accuracy: 0.0%, Avg loss: 658.162354 

Epoch 8
-------------------------------
loss: 744.509338  [    0/  118]
Test Error: 
 Accuracy: 0.0%, Avg loss: 